In [1]:
import json
import pandas as pd

In [17]:
def get_specific_info (info_name, keys, infos):
    try:
        info_index = keys.index(info_name)
        info = infos[info_index]
        return info
    except:
        return None

def try_key_var_type (dictionary):
    var_type_list = ['M', 'L', 'S']
    
    if isinstance(dictionary, dict):
        
        for var_type in var_type_list:
            value = dictionary.get(var_type, None)
            
            if not(value is None):
                return value
                
        return None
    else:
        return None

def get_additional_infos (additional_info):
    
    keys = []
    infos = []
    
    try:
        for info in additional_info:
            info = info.get('M', None) #Esse 'M' é um problema, será que pode ser outra string aqui?
            for key in info:
                
                if key == 'key':
                    dict_key = info.get(key, None)
                    
                    keys.append(try_key_var_type(dict_key)) #Mesma coisa com o 'S'
                else:
                    dict_key = info.get(key, None)
                    
                    infos.append(try_key_var_type(dict_key))

    except:
        return keys, infos

    return keys, infos
    

In [18]:

df = pd.DataFrame(columns = ['nome', 'Data de Nascimento', 'RG', 'CPF', 'email', 'telefone', 'CEP', 'sexo', 'raça', 'nível de programação', 'escolaridade'])

try:

    with open ('../dados/jason/clientes2.json', 'r', encoding = 'utf-8-sig') as read_file:
        data = json.load(read_file)

    data = data.get('Records', None) #ok

    try:
        for user in data:
            
            name = try_key_var_type(user.get('dynamodb', {}).get('NewImage', {}).get('name', None))
            cpf =  try_key_var_type(user.get('dynamodb', {}).get('NewImage', {}).get('cpf', None))
            email =  try_key_var_type(user.get('dynamodb', {}).get('NewImage', {}).get('email', None))
            
            # Accessing additionalInfo
            # additionalInfo is a dict with a single key ('L'), that contains a list with all the information we want
            additional_info = user.get('dynamodb', {}).get('NewImage', {}).get('additionalInfo', {}).get('L', {}) #Acessing additionalInfo
  
            keys, infos = get_additional_infos(additional_info)

            born_date = get_specific_info('Data de Nascimento', keys, infos)
            rg = get_specific_info('RG', keys, infos)
            phone_number = get_specific_info('Telefone', keys, infos)
            cep = get_specific_info('CEP', keys, infos)
            gender = get_specific_info('Gênero', keys, infos)
            race = get_specific_info('Etnia', keys, infos)
            progamming_exp = get_specific_info('Assinale a alternativa mais próxima ao seu conhecimento em programação*', keys, infos)
            schooling = get_specific_info('Assinale a alternativa mais próxima com relação à sua formação*', keys, infos)

            df_new_line = [name, born_date, rg, cpf, email, phone_number, cep, gender, race, progamming_exp, schooling]
            df_length = len(df)
            df.loc[df_length] = df_new_line

    except:
        print('No users or empty File')
except:
    print('File not Found or corrupted')

df

,nome,Data de Nascimento,RG,CPF,email,telefone,CEP,sexo,raça,nível de programação,escolaridade
0,Candidato 78,10/04/1999,999999999,107.089.727-21,candidato78@test.com,(99) 99999-9999,00000-000,Homem Cis,Pardo,Já programei um pouco seguindo tutoriais guiados,Não tenho (ou não estou em) formação superior
1,None,10/04/1999,999999999,107.089.727-21,candidato78@test.com,(99) 99999-9999,00000-000,Homem Cis,Pardo,Já programei um pouco seguindo tutoriais guiados,Não tenho (ou não estou em) formação superior
2,None,10/04/1999,999999999,107.089.727-21,candidato78@test.com,(99) 99999-9999,00000-000,Homem Cis,Pardo,Já programei um pouco seguindo tutoriais guiados,Não tenho (ou não estou em) formação superior
3,None,10/04/1999,999999999,107.089.727-21,candidato78@test.com,(99) 99999-9999,00000-000,Homem Cis,Pardo,Já programei um pouco seguindo tutoriais guiados,Não tenho (ou não estou em) formação superior
4,None,10/04/1999,999999999,107.089.727-21,candidato78@test.com,(99) 99999-9999,00000-000,Homem Cis,Pardo,Já programei um pouco seguindo tutoriais guiados,Não tenho (ou não estou em) formação superior
